# Google Earth Engine Component

## Initialize

In [1]:
#Import required libraries
import ee
import geemap
import math

In [2]:
#Initialize Google Earth Engine
#ee.Authenticate() just needed the 1st time
ee.Initialize()

In [3]:
# Check if geemap is working as intended - plot the leaflet map
Map = geemap.Map()

## Load Feature Collection - Shapefiles

In [4]:
#Data loads

#loads feature collection data from Google Earth Engine - We can also upload other feature collections
counties = ee.FeatureCollection("TIGER/2018/Counties")

#filter LA County
la_county = counties.filter(ee.Filter.eq('NAME', 'Los Angeles'))
sd_county = counties.filter(ee.Filter.eq('NAME', 'San Diego'))

In [5]:
#loads cities data - Not in current use

#load California Cities Shapefile
cities = ee.FeatureCollection("projects/california-lawn-detection/assets/Cities2015")

# Filter for LA City
la_cities = cities.filter(ee.Filter.eq('County', 'Los Angeles'))
sd_cities = cities.filter(ee.Filter.eq('County', 'San Diego'))


# Filter sea
la_cities_land = la_cities.filter(ee.Filter.notEquals('Notes', '3 nautical mile offshore'))
sd_cities_land = sd_cities.filter(ee.Filter.notEquals('Notes', '3 nautical mile offshore'))

sd_city = sd_cities_land.filter((ee.Filter.eq('NAME', 'San Diego')))

# How to add a layer to the base map
#Map.addLayer(la_cities_land,{}, 'la_cities')
#Map.addLayer(sd_city,{}, 'sd_city')

In [6]:
#city_names = []
#shape_list = []

#for i in range(len(la_cities_land.getInfo().get('features'))):
#    city_names.append(la_cities_land.getInfo().get('features')[i].get('properties').get('NAME'))
#    shape_list.append(la_cities_land.getInfo().get('features')[i].get('geometry'))
    

In [7]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [8]:
#loads parcel data

la_parcel_shape = ee.FeatureCollection("projects/california-lawn-detection/assets/LA_County_Parcels_Shape")

# Map.addLayer(la_parcel_shape,{'color': '00000000'}, 'la_parcel_shape')

In [9]:
#Income Data

la_county_income = ee.FeatureCollection("projects/california-lawn-detection/assets/lacountyincome-final")
#
#Map.addLayer(la_county_income,{}, 'LA County Income')

In [10]:
la_county_income2 = ee.FeatureCollection("projects/california-lawn-detection/assets/lacountyincome_update")
la_county_income = la_county_income2.select(ee.List(['Name', 'Descriptio', 'Ranking']), ee.List(['Name', 'Median_Income', 'Ranking']))
#Map.addLayer(la_county_income,{}, 'LA County Income')

In [11]:
Map

Map(bottom=754.0, center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

## Load NAIP Imagery

In [12]:
#loads NAIP 2020 and 2010 dataset and selects bands Red, Green, Blue, Near Infrared (R,G,B,N)

# rgbn imagery from 2010 to 2020
image_2020_rgbn = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2020-01-01', '2020-12-31')
    .select(['R', 'G', 'B', 'N'])
)

image_2018_rgbn = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2018-01-01', '2018-12-31')
    .select(['R', 'G', 'B', 'N'])
)

image_2016_rgbn = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2016-01-01', '2016-12-31')
    .select(['R', 'G', 'B', 'N'])
)

image_2014_rgbn = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2014-01-01', '2014-12-31')
    .select(['R', 'G', 'B', 'N'])
)

image_2012_rgbn = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2012-01-01', '2012-12-31')
    .select(['R', 'G', 'B', 'N'])
)

image_2010_rgbn = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2010-01-01', '2010-12-31')
    .select(['R', 'G', 'B', 'N'])
)

# nrg imagery from 2010 to 2020
image_2020_nrg = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2020-01-01', '2020-12-31')
    .select(['N','R','G'])
)

image_2018_nrg = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2018-01-01', '2018-12-31')
    .select(['N','R','G'])
)

image_2016_nrg = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2016-01-01', '2016-12-31')
    .select(['N','R','G'])
)

image_2014_nrg = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2014-01-01', '2014-12-31')
    .select(['N','R','G'])
)

image_2012_nrg = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2012-01-01', '2012-12-31')
    .select(['N','R','G'])
)

image_2010_nrg = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2010-01-01', '2010-12-31')
    .select(['N','R','G'])
)

#Calculate median so it can be clipped - turns image collection into an image
NAIP_2020_rgbn = image_2020_rgbn.median()
NAIP_2018_rgbn = image_2018_rgbn.median()
NAIP_2016_rgbn = image_2016_rgbn.median()
NAIP_2014_rgbn = image_2014_rgbn.median()
NAIP_2012_rgbn = image_2012_rgbn.median()
NAIP_2010_rgbn = image_2010_rgbn.median()

NAIP_2020_nrg = image_2020_nrg.median()
NAIP_2018_nrg = image_2018_nrg.median()
NAIP_2016_nrg = image_2016_nrg.median()
NAIP_2014_nrg = image_2014_nrg.median()
NAIP_2012_nrg = image_2012_nrg.median()
NAIP_2010_nrg = image_2010_nrg.median()

#Clip images to desired polygon in this case la_county

NAIP_2020_rgbn_lacounty = NAIP_2020_rgbn.clip(la_county)
NAIP_2020_nrg_lacounty = NAIP_2020_nrg.clip(la_county)
NAIP_2018_rgbn_lacounty = NAIP_2018_rgbn.clip(la_county)
NAIP_2018_nrg_lacounty = NAIP_2018_nrg.clip(la_county)
NAIP_2016_rgbn_lacounty = NAIP_2016_rgbn.clip(la_county)
NAIP_2016_nrg_lacounty = NAIP_2016_nrg.clip(la_county)
NAIP_2014_rgbn_lacounty = NAIP_2014_rgbn.clip(la_county)
NAIP_2014_nrg_lacounty = NAIP_2014_nrg.clip(la_county)
NAIP_2012_rgbn_lacounty = NAIP_2012_rgbn.clip(la_county)
NAIP_2012_nrg_lacounty = NAIP_2012_nrg.clip(la_county)
NAIP_2010_rgbn_lacounty = NAIP_2010_rgbn.clip(la_county)
NAIP_2010_nrg_lacounty = NAIP_2010_nrg.clip(la_county)


NAIP_2020_rgbn_sd_city = NAIP_2020_rgbn.clip(sd_city)
NAIP_2020_nrg_sd_city = NAIP_2020_nrg.clip(sd_city)



In [13]:
#Calculate NDVI for each image
ndvi_2020 = NAIP_2020_rgbn_lacounty.normalizedDifference(['N', 'R']).rename('NAIP_2020_NDVI')
ndvi_2018 = NAIP_2018_rgbn_lacounty.normalizedDifference(['N', 'R']).rename('NAIP_2018_NDVI')
ndvi_2016 = NAIP_2016_rgbn_lacounty.normalizedDifference(['N', 'R']).rename('NAIP_2016_NDVI')
ndvi_2014 = NAIP_2014_rgbn_lacounty.normalizedDifference(['N', 'R']).rename('NAIP_2014_NDVI')
ndvi_2012 = NAIP_2012_rgbn_lacounty.normalizedDifference(['N', 'R']).rename('NAIP_2012_NDVI')
ndvi_2010 = NAIP_2010_rgbn_lacounty.normalizedDifference(['N', 'R']).rename('NAIP_2010_NDVI')

In [14]:
#Add it as a layer
#Map.addLayer(ndvi_2020, {min : -1, max : 1, 'palette' : ['blue', 'white', 'green']}, 'NAIP_2020_NDVI')

# 2018
# Map.addLayer(ndvi_2018, {min : -1, max : 1, 'palette' : ['blue', 'white', 'green']}, 'NAIP_2018_NDVI')

# 2016
# Map.addLayer(ndvi_2016, {min : -1, max : 1, 'palette' : ['blue', 'white', 'green']}, 'NAIP_2016_NDVI')

# 2014
# Map.addLayer(ndvi_2014, {min : -1, max : 1, 'palette' : ['blue', 'white', 'green']}, 'NAIP_2014_NDVI')

# 2012
# Map.addLayer(ndvi_2012, {min : -1, max : 1, 'palette' : ['blue', 'white', 'green']}, 'NAIP_2012_NDVI')

# 2010
# Map.addLayer(ndvi_2010, {min : -1, max : 1, 'palette' : ['blue', 'white', 'green']}, 'NAIP_2010_NDVI')

In [15]:
#Add layer
Map.addLayer(NAIP_2020_rgbn_lacounty,{}, 'NAIP_2020_rgbn')
Map.addLayer(NAIP_2020_nrg_lacounty,{}, 'NAIP_2020_nrg')

Map.addLayer(NAIP_2020_rgbn_sd_city,{}, 'NAIP_2020_rgbn_sd')
Map.addLayer(NAIP_2020_nrg_sd_city,{}, 'NAIP_2020_rgbn_sd')


#2018
Map.addLayer(NAIP_2018_rgbn_lacounty,{}, 'NAIP_2018_rgbn')
Map.addLayer(NAIP_2018_nrg_lacounty,{}, 'NAIP_2018_nrg')

#2016
Map.addLayer(NAIP_2016_rgbn_lacounty,{}, 'NAIP_2016_rgbn')
Map.addLayer(NAIP_2016_nrg_lacounty,{}, 'NAIP_2016_nrg')

#2014
Map.addLayer(NAIP_2014_rgbn_lacounty,{}, 'NAIP_2014_rgbn')
Map.addLayer(NAIP_2014_nrg_lacounty,{}, 'NAIP_2014_nrg')

#2012
Map.addLayer(NAIP_2012_rgbn_lacounty,{}, 'NAIP_2012_rgbn')
Map.addLayer(NAIP_2012_nrg_lacounty,{}, 'NAIP_2012_nrg')

#2010
Map.addLayer(NAIP_2010_rgbn_lacounty,{}, 'NAIP_2010_rgbn')
Map.addLayer(NAIP_2010_nrg_lacounty,{}, 'NAIP_2010_nrg')

## Texture

In [16]:
def add_NDVI(image):
    image = image.addBands(image.normalizedDifference(['N','R']).rename('NDVI'))
    return image

In [17]:
NAIP_2020_rgbn_ndvi_lacounty = add_NDVI(NAIP_2020_rgbn_lacounty)

NAIP_2020_rgbn_ndvi_sd_city = add_NDVI(NAIP_2020_rgbn_sd_city)

NAIP_2018_rgbn_ndvi_lacounty = add_NDVI(NAIP_2018_rgbn_lacounty)
NAIP_2016_rgbn_ndvi_lacounty = add_NDVI(NAIP_2016_rgbn_lacounty)
NAIP_2014_rgbn_ndvi_lacounty = add_NDVI(NAIP_2014_rgbn_lacounty)
NAIP_2012_rgbn_ndvi_lacounty = add_NDVI(NAIP_2012_rgbn_lacounty)
NAIP_2010_rgbn_ndvi_lacounty = add_NDVI(NAIP_2010_rgbn_lacounty)

In [18]:
def neighborhood_bands(image, band):
    i_8_bit = image.select(band).toUint8()
    square = ee.Kernel.square(**{'radius': 4})
    entropy = i_8_bit.entropy(square)
    glcm = i_8_bit.glcmTexture(**{'size': 4})
    contrast = glcm.select(str(band)+'_contrast')
    
    # Create a list of weights for a 9x9 kernel.
    list = [1, 1, 1, 1, 1, 1, 1, 1, 1]
    # The center of the kernel is zero.
    centerList = [1, 1, 1, 1, 0, 1, 1, 1, 1]
    # Assemble a list of lists: the 9x9 kernel weights as a 2-D matrix.
    lists = [list, list, list, list, centerList, list, list, list, list]
    # Create the kernel from the weights.
    # Non-zero weights represent the spatial neighborhood.
    kernel = ee.Kernel.fixed(9, 9, lists, -4, -4, False)
    neighs = i_8_bit.neighborhoodToBands(kernel)
    gearys = i_8_bit.subtract(neighs).pow(2).reduce(ee.Reducer.sum()).divide(math.pow(9, 2))
    image = image.addBands(entropy.rename(str(band)+'_Entropy')).addBands(contrast.rename(str(band)+'_Contrast')).addBands(gearys.rename(str(band)+'_Gearys'))   
    return image


In [19]:
bands = ['R', 'G', 'B', 'N', 'NDVI']
for band in bands:
    NAIP_2020_rgbn_ndvi_lacounty = neighborhood_bands(NAIP_2020_rgbn_ndvi_lacounty,band)
    
    NAIP_2020_rgbn_ndvi_sd_city = neighborhood_bands(NAIP_2020_rgbn_ndvi_sd_city,band)
    
    NAIP_2018_rgbn_ndvi_lacounty = neighborhood_bands(NAIP_2018_rgbn_ndvi_lacounty,band)
    NAIP_2016_rgbn_ndvi_lacounty = neighborhood_bands(NAIP_2016_rgbn_ndvi_lacounty,band)
    NAIP_2014_rgbn_ndvi_lacounty = neighborhood_bands(NAIP_2014_rgbn_ndvi_lacounty,band)
    NAIP_2012_rgbn_ndvi_lacounty = neighborhood_bands(NAIP_2012_rgbn_ndvi_lacounty,band)
    NAIP_2010_rgbn_ndvi_lacounty = neighborhood_bands(NAIP_2010_rgbn_ndvi_lacounty,band)

## Load Labeled Data

In [20]:
## Loading feature collections from Google Earth Engine

water_1 = ee.FeatureCollection("projects/california-lawn-detection/assets/water_torrance_0610")
water_2 = ee.FeatureCollection("projects/california-lawn-detection/assets/water_torrance_0701_400")
vegetation_trees = ee.FeatureCollection("projects/california-lawn-detection/assets/trees_torrance")
vegetation_grass = ee.FeatureCollection("projects/california-lawn-detection/assets/grass_torrance").limit(400)
turf_1 = ee.FeatureCollection("projects/california-lawn-detection/assets/turf_torrance1")
turf_2 = ee.FeatureCollection("projects/california-lawn-detection/assets/turf_torrance2")
impervious_1 = ee.FeatureCollection("projects/california-lawn-detection/assets/impervious_torrance1").limit(35)
impervious_2 = ee.FeatureCollection("projects/california-lawn-detection/assets/impervious_torrance2").limit(35)
soil = ee.FeatureCollection("projects/california-lawn-detection/assets/soil_reduced_070222")

water = water_1.merge(water_2)
turf = turf_1.merge(turf_2)
impervious= impervious_1.merge(impervious_2)

In [21]:
def conditional_water(feat):
    return ee.Algorithms.If(ee.Number(feat.get('landcover')).eq(1),feat.set({'landcover': 0}),feat)

def conditional_trees(feat):
    return ee.Algorithms.If(ee.Number(feat.get('landcover')).eq(2),feat.set({'landcover': 1}),feat)

def conditional_grass(feat):
    return ee.Algorithms.If(ee.Number(feat.get('landcover')).eq(3),feat.set({'landcover': 2}),feat)

def conditional_turf(feat):
    return ee.Algorithms.If(ee.Number(feat.get('landcover')).eq(4),feat.set({'landcover': 3}),feat)

def conditional_impervious(feat):
    return ee.Algorithms.If(ee.Number(feat.get('landcover')).eq(6),feat.set({'landcover': 4}),feat)

def conditional_soil(feat):
    return ee.Algorithms.If(ee.Number(feat.get('landcover')).eq(7),feat.set({'landcover': 5}),feat)

water_tr = water.map(conditional_water)
trees_tr = vegetation_trees.map(conditional_trees)
grass_tr = vegetation_grass.map(conditional_grass)
turf_tr = turf.map(conditional_turf)
impervious_tr = impervious.map(conditional_impervious)
soil_tr = soil.map(conditional_soil)

labeled_set = water_tr.merge(trees_tr).merge(grass_tr).merge(turf_tr).merge(impervious_tr).merge(soil_tr)

In [22]:
water_test = ee.FeatureCollection("projects/california-lawn-detection/assets/water_test")
vegetation_trees_test = ee.FeatureCollection("projects/california-lawn-detection/assets/trees_test")
vegetation_grass_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/grass_test")
turf_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/turf_test")
impervious_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/impervious_reduced_test")
soil_test  = ee.FeatureCollection("projects/california-lawn-detection/assets/soil_reduced_070222")

test_set = water_test.merge(vegetation_trees_test).merge(vegetation_grass_test).merge(turf_test).merge(impervious_test).merge(soil_test)

## Building Training Set

In [23]:
# Overlay the points on the imagery to get training.
label = 'landcover'
bands = ['R', 'G', 'B', 'N', 'NDVI', 'N_Entropy', 'N_Contrast','N_Gearys']
#         'R_Entropy',
#         'R_Contrast',
#         'R_Gearys',
#         'G_Entropy',
#         'G_Contrast',
#         'G_Gearys',
#         'B_Entropy',
#         'B_Contrast',
#         'B_Gearys']

training2020 = NAIP_2020_rgbn_ndvi_lacounty.select(bands).sampleRegions(**{
  'collection': labeled_set,
  'properties': [label],
  'scale': 1
})

In [24]:
test2020 = NAIP_2020_rgbn_ndvi_lacounty.select(bands).sampleRegions(**{
  'collection': test_set,
  'properties': [label],
  'scale': 1
})

In [25]:
print("Training Set Size in Pixels", training2020.aggregate_count('R').getInfo())

Training Set Size in Pixels 248876


In [26]:
print("Test Set Size in Pixels", test2020.aggregate_count('R').getInfo())

Test Set Size in Pixels 15574


### Machine Learning Model

In [28]:
trained2020 = ee.Classifier.smileRandomForest(numberOfTrees = 230, minLeafPopulation = 50, bagFraction= 0.6).train(training2020, label, bands)

In [30]:
classified2020 = NAIP_2020_rgbn_ndvi_lacounty.select(bands).classify(trained2020)

In [39]:
classified2020_sd = NAIP_2020_rgbn_ndvi_sd_city.select(bands).classify(trained2020)

In [34]:
classified2018 = NAIP_2018_rgbn_ndvi_lacounty.select(bands).classify(trained2020)

In [35]:
classified2010 = NAIP_2010_rgbn_ndvi_lacounty.select(bands).classify(trained2020)

In [31]:
legend_keys = ['water', 'vegetation_trees', 'vegetation_grass', 'turf','impervious','soil']
#legend_colors = ['#2252ff', '#086f69', '#1dd12d', '#0b9ed5' ,' #7c807c','#a47041']
legend_colors = ['#0B6AEF', '#097407', '#0CE708', '#8C46D2' ,' #A1A8AF','#D47911']

Map.addLayer(classified2020.select(['classification']), {'min': 0, 'max': 5, 'palette': legend_colors}, 'Classification2020')

In [42]:
Map.addLayer(classified2020_sd.select(['classification']), {'min': 0, 'max': 5, 'palette': legend_colors}, 'Classification2020_sd')

In [36]:
Map.addLayer(classified2018.select(['classification']), {'min': 0, 'max': 5, 'palette': legend_colors}, 'Classification2018')

In [37]:
Map.addLayer(classified2010.select(['classification']), {'min': 0, 'max': 5, 'palette': legend_colors}, 'Classification2010')

In [38]:
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')

The legend colors must be a list of tuples.


In [32]:
Map

Map(bottom=754.0, center=[19.973348786110613, -0.17578125], controls=(WidgetControl(options=['position', 'tran…

## Evaluation Metrics

In [40]:
train_accuracy = trained2020.confusionMatrix()
train_accuracy.getInfo()

[[2157, 11, 21, 2, 145, 0],
 [0, 64872, 3172, 10, 322, 37],
 [0, 6198, 88115, 8, 292, 128],
 [0, 274, 42, 2838, 136, 0],
 [7, 98, 74, 16, 70778, 128],
 [0, 26, 121, 1, 290, 8557]]

In [41]:
import numpy as np
train_cm = np.array(train_accuracy.getInfo())

In [42]:
train_cm

array([[ 2157,    11,    21,     2,   145,     0],
       [    0, 64872,  3172,    10,   322,    37],
       [    0,  6198, 88115,     8,   292,   128],
       [    0,   274,    42,  2838,   136,     0],
       [    7,    98,    74,    16, 70778,   128],
       [    0,    26,   121,     1,   290,  8557]])

In [43]:
def f1(C):
    num_classes = np.shape(C)[0]
    f1_score = np.zeros(shape=(num_classes,), dtype='float32')
#    weights = np.sum(C, axis=0)/np.sum(C)

    for j in range(num_classes):
        tp = np.sum(C[j, j])
        fp = np.sum(C[j, np.concatenate((np.arange(0, j), np.arange(j+1, num_classes)))])
        fn = np.sum(C[np.concatenate((np.arange(0, j), np.arange(j+1, num_classes))), j])
#         tn = np.sum(C[np.concatenate((np.arange(0, j), np.arange(j+1, num_classes))), np.concatenate((np.arange(0, j), np.arange(j+1, num_classes)))])

        precision = tp/(tp+fp) if (tp+fp) > 0 else 0
        recall = tp/(tp+fn) if (tp+fn) > 0 else 0
        #f1_score[j] = 2*precision*recall/(precision + recall)*weights[j] if (precision + recall) > 0 else 0
        f1_score[j] = 2*precision*recall/(precision + recall) if (precision + recall) > 0 else 0
        store = f1_score

    f1_score = np.mean(f1_score)
    return f1_score, store

In [44]:
f1(train_cm)

(0.95022005,
 array([0.9586667 , 0.92745835, 0.9460185 , 0.92068124, 0.9894593 ,
        0.9590362 ], dtype=float32))

In [45]:
train_accuracy.accuracy().getInfo()

0.9535551841077484

In [46]:
train_accuracy.kappa().getInfo()

0.933354924074784

In [47]:
test_accuracy = trained2020.confusionMatrix()
train_accuracy.getInfo()

[[2157, 11, 21, 2, 145, 0],
 [0, 64872, 3172, 10, 322, 37],
 [0, 6198, 88115, 8, 292, 128],
 [0, 274, 42, 2838, 136, 0],
 [7, 98, 74, 16, 70778, 128],
 [0, 26, 121, 1, 290, 8557]]

### Test Accuracy

In [48]:
validated = test2020.classify(trained2020)

In [49]:
test_accuracy = validated.errorMatrix('landcover', 'classification')

In [50]:
test_accuracy.getInfo()

[[275, 0, 0, 0, 0, 0],
 [0, 2592, 174, 2, 32, 1],
 [0, 307, 1213, 0, 0, 12],
 [0, 63, 3, 772, 22, 0],
 [0, 10, 0, 2, 1047, 44],
 [0, 26, 121, 1, 290, 8565]]

In [51]:
test_cm = np.array(test_accuracy.getInfo())

In [52]:
f1(test_cm)

(0.9076509,
 array([1.        , 0.89394724, 0.79723954, 0.9431888 , 0.83961505,
        0.9719149 ], dtype=float32))

In [53]:
test_accuracy.accuracy().getInfo()

0.9287273661230255

In [54]:
test_accuracy.kappa().getInfo()

0.886220341163507

### Experimental Section

In [55]:
la_county_income_zipcode2 = ee.FeatureCollection("projects/california-lawn-detection/assets/income_zipcode2019")
la_county_income_zipcode = la_county_income_zipcode2.select(ee.List(['zipcode', '2019zipcod','shape_area']), ee.List(['ZipCode', 'Median_Income','Area_sqft']))

z90802 = la_county_income_zipcode.filter(ee.Filter.eq('ZipCode', '90802')) 

## Binary Classification and Area Calculation

In [59]:
def area_calculation(image, class_number, shape, pixel_scale):
    
    areaImage = image.eq(class_number).multiply(ee.Image.pixelArea())
    
    area = areaImage.reduceRegion(
        reducer = ee.Reducer.sum(), 
        geometry = shape.geometry(), 
        scale = pixel_scale, 
        maxPixels = 1e13)
    
    area_sq_m = area.getInfo().get('classification')
    
    #area_sq_km = area_sq_m / 1e6
    
    return area_sq_m

In [45]:
def ndvi_calculation(image, class_number, shape, pixel_scale):
    
    areaImage = image.eq(2).select('NDVI')
    
    area = areaImage.reduceRegion(
        reducer = ee.Reducer.count(), 
        geometry = shape.geometry(), 
        scale = pixel_scale, 
        maxPixels = 1e13)
    
    return area

In [46]:
Map

Map(bottom=754.0, center=[46.529, 6.746], controls=(WidgetControl(options=['position', 'transparent_bg'], widg…

In [47]:
def ndvi_calculation_2(image, class_number, shape, pixel_scale):
    
    areaImage = image.select('NDVI').eq(class_number)
    
    minMaxValues = areaImage.reduceRegion(reducer = ee.Reducer.minMax(),
                                 geometry = shape.geometry())
    return minMaxValues

In [ ]:
water_area = area_calculation(classified2020, 1, la_county, 20)
vegetation_trees_area = area_calculation(classified2020, 2, la_county, 20)
vegetation_grass_area = area_calculation(classified2020, 3, la_county, 20)
turf_area = area_calculation(classified2020, 4, la_county, 20)
impervious_area = area_calculation(classified2020, 6, la_county, 20)
soil_area = area_calculation(classified2020, 7, la_county, 20)

total_area = water_area + vegetation_trees_area + vegetation_grass_area + turf_area + impervious_area + soil_area

water_percentage = water_area / total_area
vegetation_trees_percentage = vegetation_trees_area / total_area
vegetation_grass_percentage = vegetation_grass_area / total_area
turf_percentage = turf_area / total_area
impervious_percentage = impervious_area / total_area
soil_percentage = soil_area / total_area

In [ ]:
print(water_area)
print(vegetation_trees_area)

In [52]:
#works
la_county_income_zipcode2 = ee.FeatureCollection("projects/california-lawn-detection/assets/income_zipcode2019")
la_county_income_zipcode = la_county_income_zipcode2.select(ee.List(['zipcode', '2019zipcod','shape_area']), ee.List(['ZipCode', 'Median_Income','Area_sqft']))
z90802 = la_county_income_zipcode.filter(ee.Filter.eq('ZipCode', '90802')) 

classified2020_clipped = classified2020.clip(z90802)

In [122]:
#works
NDVIfortress = ndvi_2020.updateMask(classified2020_clipped.select('classification').eq(1))
Map.addLayer(NDVIfortress)

In [123]:
Map

Map(bottom=419919.0, center=[33.787422527153296, -118.14456939697267], controls=(WidgetControl(options=['posit…

In [124]:
#works
areaImage = NDVIfortress
    
area = areaImage.reduceRegion(
    reducer = ee.Reducer.mean(), 
    geometry = z90802.geometry(), 
    scale = 1, 
    maxPixels = 1e13)

In [126]:
#works
print(area.getInfo())

{'NAIP_2020_NDVI': 0.33721675193349554}


In [112]:
ndvi_2020_clipped = ndvi_2020.clip(z90802)

In [113]:
Map.addLayer(
    ndvi_2020_clipped, {min:0, max:1},
    'ndvi_2020_clipped')
Map

Map(bottom=3357823.0, center=[33.767054120314754, -118.19353580474855], controls=(WidgetControl(options=['posi…

In [110]:
ndvi_2020_clipped_2 = ndvi_2020_clipped.clip()

In [27]:
# Put the EEified model in the appropriate bucket and API name
PROJECT = 'w210-351617'
OUTPUT_BUCKET = 'test-tf-gee'
EEIFIED_DIR = 'gs://' + OUTPUT_BUCKET + '/eeified_pixel_model'

MODEL_NAME = 'CNN_Nbands_model'
VERSION_NAME = 'v0'
REGION = 'us-central1'

training_classes_list_text = ['water',
                              'vegetation_trees',
                              'vegetation_grass',
                              'turf',
                              'impervious',
                              'soil']

In [28]:
# Point to the model hosted on AI Platform.  If you specified a region other
# than the default (us-central1) at model creation, specify it here.
final_model = ee.Model.fromAiPlatformPredictor(
    projectName=PROJECT,
    modelName=MODEL_NAME,
    version=VERSION_NAME,
    # Can be anything, but don't make it too big.
    inputTileSize=[12, 12],
    # # Keep this the same as your training data.
    # proj=ee.Projection('EPSG:4326').atScale(30),
    # fixInputProj=True,
    # Note the names here need to match what you specified in the
    # output dictionary you passed to the EEifier.
    outputBands={'output': {
        'type': ee.PixelType.float(),
        'dimensions': 1
      }
    },
)

In [29]:
# Get predictions

la_image_2020 = NAIP_2020_rgbn_ndvi_lacounty.select(bands)
la_image_2018 = NAIP_2018_rgbn_ndvi_lacounty.select(bands)
la_image_2016 = NAIP_2016_rgbn_ndvi_lacounty.select(bands)
la_image_2014 = NAIP_2014_rgbn_ndvi_lacounty.select(bands)
la_image_2012 = NAIP_2012_rgbn_ndvi_lacounty.select(bands)
la_image_2010 = NAIP_2010_rgbn_ndvi_lacounty.select(bands)

In [30]:
# Get predictions
predictions_2020 = final_model.predictImage(la_image_2020.float().toArray())
probabilities_2020 = predictions_2020.arrayFlatten([training_classes_list_text])
label_2020 = predictions_2020.arrayArgmax().arrayGet([0]).rename('label')

In [31]:
predictions_2018 = final_model.predictImage(la_image_2018.float().toArray())
probabilities_2018 = predictions_2018.arrayFlatten([training_classes_list_text])
label_2018 = predictions_2018.arrayArgmax().arrayGet([0]).rename('label')

In [32]:
predictions_2016 = final_model.predictImage(la_image_2016.float().toArray())
probabilities_2016 = predictions_2016.arrayFlatten([training_classes_list_text])
label_2016 = predictions_2016.arrayArgmax().arrayGet([0]).rename('label')

In [33]:
predictions_2014 = final_model.predictImage(la_image_2014.float().toArray())
probabilities_2014 = predictions_2014.arrayFlatten([training_classes_list_text])
label_2014 = predictions_2014.arrayArgmax().arrayGet([0]).rename('label')

In [34]:
predictions_2012 = final_model.predictImage(la_image_2012.float().toArray())
probabilities_2012 = predictions_2012.arrayFlatten([training_classes_list_text])
label_2012 = predictions_2012.arrayArgmax().arrayGet([0]).rename('label')

In [35]:
predictions_2010 = final_model.predictImage(la_image_2010.float().toArray())
probabilities_2010 = predictions_2010.arrayFlatten([training_classes_list_text])
label_2010 = predictions_2010.arrayArgmax().arrayGet([0]).rename('label')

In [36]:
legend_keys = ['water', 'vegetation_trees', 'vegetation_grass', 'turf','impervious','soil']
legend_colors = ['#0B6AEF', '#097407', '#0CE708', '#8C46D2' ,' #A1A8AF','#D47911']

# Map.addLayer(predictions, {'min': 0, 'max': 5}, 'Classification')

Map.addLayer(label_2020,{'min': 0, 'max': 5, 'palette': legend_colors},"NN_2020")
Map.addLayer(label_2018,{'min': 0, 'max': 5, 'palette': legend_colors},"NN_2018")
Map.addLayer(label_2016,{'min': 0, 'max': 5, 'palette': legend_colors},"NN_2016")
Map.addLayer(label_2014,{'min': 0, 'max': 5, 'palette': legend_colors},"NN_2014")
Map.addLayer(label_2012,{'min': 0, 'max': 5, 'palette': legend_colors},"NN_2012")
Map.addLayer(label_2010,{'min': 0, 'max': 5, 'palette': legend_colors},"NN_2010")

In [37]:
Map

Map(bottom=754.0, center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…